In [1]:
path = 'raw_data//'
def read_files(path):
    """Create a list of files, read them and return a concatenated dataframe."""    
    import pandas as pd
    import os

    files = os.listdir(path)
    df = (pd.read_csv(path + f, sep=';',
                      encoding='latin-1',
                      usecols=['DT_REFER', 'DENOM_CIA', 'ORDEM_EXERC',
                               'CD_CVM', 'CD_CONTA', 
                               'DS_CONTA',
                               'VL_CONTA'], parse_dates=['DT_REFER'],
                      infer_datetime_format=True) for f in files)
    df = pd.concat(df).reindex()
    return df[df.ORDEM_EXERC == 'ÚLTIMO']

In [2]:
df = read_files(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1727833 entries, 1 to 25653
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   DT_REFER     datetime64[ns]
 1   DENOM_CIA    object        
 2   CD_CVM       int64         
 3   ORDEM_EXERC  object        
 4   CD_CONTA     object        
 5   DS_CONTA     object        
 6   VL_CONTA     float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 105.5+ MB


In [3]:
def pivot_df(df, length=4):
    """Get the dataframe pivoted."""
    df = df[(df.CD_CONTA.str.len() <= length)]
    # Getting last layout as a model
    cod_conta = df.loc[df.DT_REFER == max(df.DT_REFER),
                      ['CD_CONTA']]
    df = df[df.CD_CONTA.isin(cod_conta.CD_CONTA)]
    df_pivoted = df.pivot_table(index=['DT_REFER', 'CD_CVM'], columns=['CD_CONTA'], values=['VL_CONTA'])
    df_pivoted.columns = df_pivoted.columns.droplevel()
    return df_pivoted.dropna(axis=1, how='all')

In [4]:
def cod_items(df, length=4):
    df = df[(df.CD_CONTA.str.len() <= length)]
    return df[['CD_CONTA',
               'DS_CONTA']].set_index('CD_CONTA')['DS_CONTA'].sort_index().to_dict()

In [12]:
def cod_names():
    return df[['CD_CVM',
               'DENOM_CIA']].set_index('CD_CVM')['DENOM_CIA'].sort_index().to_dict()

In [5]:
df_pivoted = pivot_df(df, length=4)
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6379 entries, (Timestamp('2010-12-31 00:00:00'), 94) to (Timestamp('2019-12-31 00:00:00'), 80179)
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       6379 non-null   float64
 1   1.01    6379 non-null   float64
 2   1.02    6379 non-null   float64
 3   1.03    414 non-null    float64
 4   1.04    194 non-null    float64
 5   1.05    194 non-null    float64
 6   1.06    194 non-null    float64
 7   1.07    194 non-null    float64
 8   2       6379 non-null   float64
 9   2.01    6379 non-null   float64
 10  2.02    6379 non-null   float64
 11  2.03    6379 non-null   float64
 12  2.04    194 non-null    float64
 13  2.05    414 non-null    float64
 14  2.06    194 non-null    float64
 15  2.07    194 non-null    float64
 16  2.08    194 non-null    float64
 17  3.01    6372 non-null   float64
 18  3.02    6372 non-null   float64
 19  3.03    6372 non-null   float64
 20  

In [6]:
cod_items(df, length=4)

{'1': 'Ativo Total',
 '1.01': 'Ativo Circulante',
 '1.02': 'Ativo Não Circulante',
 '1.03': 'Ativo Permanente',
 '1.04': 'Tributos Diferidos',
 '1.05': 'Outros Ativos',
 '1.06': 'Investimentos',
 '1.07': 'Imobilizado',
 '1.08': 'Intangível',
 '2': 'Passivo Total',
 '2.01': 'Passivo Circulante',
 '2.02': 'Passivo Não Circulante',
 '2.03': 'Patrimônio Líquido Consolidado',
 '2.04': 'Provisões',
 '2.05': 'Passivos Fiscais',
 '2.06': 'Outros Passivos',
 '2.07': 'Passivos sobre Ativos Não Correntes a Venda e Descontinuados',
 '2.08': 'Patrimônio Líquido Consolidado',
 '3.01': 'Receita de Venda de Bens e/ou Serviços',
 '3.02': 'Custo dos Bens e/ou Serviços Vendidos',
 '3.03': 'Resultado Bruto',
 '3.04': 'Despesas/Receitas Operacionais',
 '3.05': 'Resultado Antes do Resultado Financeiro e dos Tributos',
 '3.06': 'Resultado Financeiro',
 '3.07': 'Resultado Antes dos Tributos sobre o Lucro',
 '3.08': 'Provisão para IR e Contribuição Social',
 '3.09': 'Resultado Líquido das Operações Continuadas

In [23]:
print(df_pivoted.sample(20).iloc[:,0:3])

CD_CONTA                     1        1.01         1.02
DT_REFER   CD_CVM                                      
2011-12-31 5983        25784.0      3107.5      22676.5
2016-12-31 14613       17600.0        37.0      17563.0
2012-12-31 22390         101.0       101.0          0.0
2013-12-31 20524     3851074.5   1744848.0    2106226.5
2010-12-31 2909       522417.0    286444.5     235972.5
2016-12-31 21520      822446.5     91631.0     730815.5
2012-12-31 22934       57217.0     57217.0          0.0
2010-12-31 20974     1257475.5    422941.0     834534.5
           5410      5523579.0   2773280.5    2750298.5
2011-12-31 5410      6461108.0   3225753.0    3235355.0
2013-12-31 6343      4338218.5   1691015.0    2647203.5
2015-12-31 22217      463843.5     55821.5     408022.0
           2437    122404301.5  21898276.5  100506025.0
2010-12-31 21059         273.0       273.0          0.0
           8451      2583033.0   1651135.5     931897.5
2014-12-31 14605     4402872.0   1368666.0    30

In [ ]:
df_pivoted['Current_Ratio'] = df_pivoted['1.01']/df_pivoted['2.01']

In [28]:
df_pivoted['1.01']/df_pivoted['2.01']

DT_REFER    CD_CVM
2010-12-31  94        2.366105
            140       1.679680
            701       5.773780
            906       1.261044
            922       0.762841
                        ...   
2019-12-31  24880     1.054207
            80020     8.745793
            80047     1.491686
            80152          NaN
            80179     1.355918
Length: 6379, dtype: float64